# LTA DataMall API User Guide

Reference document url: https://datamall.lta.gov.sg/content/dam/datamall/datasets/LTA_DataMall_API_User_Guide.pdf

In [1]:
import os,sys
PROJECT_PATH = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.insert(0, PROJECT_PATH)

In [2]:
# Import api key from api_key.py's LTA_API_KEY
from conf.api_key import LTA_API_KEY
from typing import Union, Dict
import requests
from zipfile import ZipFile

In [19]:
def api_query(api_link: str,  agent_id: str, api_key: str, params_dict: Dict = None) -> Union[Dict,None]:
    """Function which executes query via an api link using a provided agent_id as an identifier to avoid rejection of query request

    Args:
        api_link (str): API Link which requests is to be made
        agent_id (str): Id used for request header
        api_key (str): API Key provided
        params_dict (Dict): Dictionary containing parameters to be passed in requests' get method

    Returns:
        Dictionary containing request content. None when exception are encountered.
    """
    req_headers = {"User-agent": agent_id, "AccountKey": api_key, "Content-Type": "application/json"}
    try:
        res = requests.get(url=api_link,
                           params=params_dict,
                           headers=req_headers,
                           timeout=5)
        # Raise if HTTPError occured
        res.raise_for_status()

        # Check the status code before extending the number of posts
        if res.status_code == 200:
            print(f"Request successful with status code {res.status_code}")
            the_json = res.json()
            return the_json
        else:
            print(f"Return unssucessful with status code {res.status_code}")
            return res.status_code

    except requests.exceptions.HTTPError as errh:
        print(errh)
    except requests.exceptions.ConnectionError as errc:
        print(errc)
    except requests.exceptions.Timeout as errt:
        print(errt)
    except requests.exceptions.RequestException as err:
        print(err)
    return None

## List of API URLs provided by LTA Datamall
- Bus Arrival: http://datamall2.mytransport.sg/ltaodataservice/BusArrivalv2 (1min; Need Request Parameters: BusStopCode and ServiceNo)
- Bus Services: http://datamall2.mytransport.sg/ltaodataservice/BusServices (Ad hoc)
- Bus Routes: http://datamall2.mytransport.sg/ltaodataservice/BusRoutes (Ad hoc)
- Bus Stops: http://datamall2.mytransport.sg/ltaodataservice/BusStops (Ad Hoc)
- Passenger Volume by Bus Stops: http://datamall2.mytransport.sg/ltaodataservice/PV/Bus (Every 15th of mth update prev mth data)
- Passenger Volume By Origin Destination Bus Stops: http://datamall2.mytransport.sg/ltaodataservice/PV/ODBus (Every 15th of mth update prev mth data)
- Passenger Volume By Origin Destination Train Stations: http://datamall2.mytransport.sg/ltaodataservice/PV/ODTrain (Every 15th of mth update prev mth data)
- Passenger Volume By Train Stations: http://datamall2.mytransport.sg/ltaodataservice/PV/Train (Every 15th of mth update prev mth data)
- Taxi Availability: http://datamall2.mytransport.sg/ltaodataservice/Taxi-Availability (1 min)
- Taxi Stand: http://datamall2.mytransport.sg/ltaodataservice/TaxiStands (monthly)
- Train Service Alerts: http://datamall2.mytransport.sg/ltaodataservice/TrainServiceAlerts (Ad hoc)
- Carpark Availability: http://datamall2.mytransport.sg/ltaodataservice/CarParkAvailabilityv2 (1min)
- ERP Rates: http://datamall2.mytransport.sg/ltaodataservice/ERPRates (Ad hoc)
- Estimated Travel Times: http://datamall2.mytransport.sg/ltaodataservice/EstTravelTimes (5mins)
- Faulty Traffic Lights : http://datamall2.mytransport.sg/ltaodataservice/FaultyTrafficLights (2mins -whenever there are updates)
- Road Openings: http://datamall2.mytransport.sg/ltaodataservice/RoadOpenings (24 hours -whenever there are updates)
- Road Works: http://datamall2.mytransport.sg/ltaodataservice/RoadWorks (24 hours - whenever there are updates)
- Traffic Images: http://datamall2.mytransport.sg/ltaodataservice/Traffic-Imagesv2 (1 to 5 minutes)
- Traffic Incidents: http://datamall2.mytransport.sg/ltaodataservice/TrafficIncidents (2mins - whenever there are updates)
- Traffic Speed Bands: http://datamall2.mytransport.sg/ltaodataservice/v3/TrafficSpeedBands (5 mins)
- VMS/EMAS: http://datamall2.mytransport.sg/ltaodataservice/VMS (2 mins)
- Bicycle Parking: http://datamall2.mytransport.sg/ltaodataservice/BicycleParkingv2 (Need Parameters: Lat, Long, Dist: (Radius in km))
- Geospatial: http://datamall2.mytransport.sg/ltaodataservice/GeospatialWholeIslan (Ad hoc)
- Facilities Maintenance: http://datamall2.mytransport.sg/ltaodataservice/FacilitiesMaintenace (Ad hoc)
- Crowd Density Real Time: http://datamall2.mytransport.sg/ltaodataservice/PCDForecast (30 mins,Need Parameters: TrainLine)
- Crowd Density Forecast: http://datamall2.mytransport.sg/ltaodataservice/PCDRealTime (30 mins, Need Parameters: TrainLine)
- Traffic Flow: http://datamall2.mytransport.sg/ltaodataservice/TrafficFlow (Quarterly)

In [20]:
# TrainStation PassengerVolume
train_network_line_list = ["CCL",
                           "CEL",
                           "CGL",
                           "DTL",
                           "EWL",
                           "NEL",
                           "DTL",
                           "NEL",
                           "NSL",
                           "BPL",
                           "SLRT",
                           "PLRT"]

train_station_url = "http://datamall2.mytransport.sg/ltaodataservice/PV/Train"
train_station_api_response = api_query(api_link=train_station_url, agent_id="test", api_key=LTA_API_KEY, params_dict={"TrainLine": "CCL"})

response_content = train_station_api_response.get("value")[0].get("Link")
response_content

Request successful with 200


'https://ltafarecard.s3.ap-southeast-1.amazonaws.com/202312/transport_node_train_202312.zip?X-Amz-Security-Token=IQoJb3JpZ2luX2VjEJf%2F%2F%2F%2F%2F%2F%2F%2F%2F%2FwEaDmFwLXNvdXRoZWFzdC0xIkYwRAIgIF%2BljItxkhmvS20f9BbZE8Ys6Yutsywh6%2BiCct6GjJICIBeCTUCf7SzLzeY9pjRKcAizqcDUmuDKB1%2F6UNcTlex1KsIFCEAQBBoMMzQwNjQ1MzgxMzA0Igz6cDEwhw2Mejw9qHsqnwXbvgbD5XT8lwQPUGggey8CvRirlDkhARQL19PZPfIekmMUrOxEF9ni1I7wS%2BmH%2FypL0zxc5NNcqFTr8fHz0jR6svK5k6FggJuahOg2C2hTcvuITNirrdv0B6dnD%2FP9%2FdCrDu4BAhKcRWWxV8ppk0QGFLA5q%2FiytT9raAbhDM0cG1LSTLfHd2%2BGFzVaLdk672Fs8x43dwoK6OIFfkbW9O8vRH81Bt7bOFfb%2BbAEm9tXrugIGjvMCsRofJCuxMN1pkbWyifA0uXnwg%2FkoSOsvXDsH%2B5GwSw0nvneeAKG4f9ykpPvEuRfrpNiTUcKk4JMGXPXanmOecuFYNIxnOTEqDWt%2BtzUAlG8o%2BTo9%2BxPa6mYrLK38Doy0fXjtUwr%2Fqet8IAlGE%2BEBc5db%2BB23tDx1kpxphON9%2FwYM6Utpq%2B0cBfw%2Fz6ETV3LScGKXSTvpKfO9JvYFIKJ506l9v9LxMfIuyp7i5EPxfLGY60CF6TNv08X2YIrkKutNUj2Ox69xgHIC6aYclqABGvCXKsDVTLuv8aM033sfJMg0NR4h1jfgEmUw9HH6jGrb%2Fz2a1gRvU%2FWfncT4yWx4JV6hZSWHhHEj0%2FYU9oF6E0vivd8bv7ILL%2F%2

In [12]:
# TrafficFlow
traffic_flow_url = "http://datamall2.mytransport.sg/ltaodataservice/TrafficFlow"
traffic_flow_api_response = api_query(api_link=traffic_flow_url, agent_id="test", api_key=LTA_API_KEY)

response_content = traffic_flow_api_response.get("value")[0].get("Link")
response_content

Request successful with 200


'https://dmprod-datasets.s3.ap-southeast-1.amazonaws.com/traffic-flow/data/trafficflow.json?X-Amz-Security-Token=IQoJb3JpZ2luX2VjEJb%2F%2F%2F%2F%2F%2F%2F%2F%2F%2FwEaDmFwLXNvdXRoZWFzdC0xIkcwRQIgLuU3DVNl%2B32APHo6AOZS61nkfez%2Bx%2BVPotdB5FATHSwCIQDPng47R%2F1uHNGIk863oBzRWeN0TFSV6OXjp0G0IBOWlyrCBQg%2FEAQaDDM0MDY0NTM4MTMwNCIMHtuC3g5UfwmSzbDPKp8FLZ1XSbbp8wSC4qMS6Mir3ucRughq9QbgSxzqWspHUlcKHJTH4woaDBDAsjFi7eznouf%2BWMWNKctFLWMZDjKQX2SdHlsbNg3OF7L0oPYkLCD%2BqFOgv75zjTQ2%2Bv1d%2FUPxDIi6Z6A568OH6Sf%2FqQL4P7xfFKvnQswDYT9tuctFFFELmdzCsOe3LvoKcUduH3R%2B3Li3T5u998y0U578acYVhiUFWJk2XOqiXDe%2Bz64f0pc38apveHZEpyid85LpGsN%2BUj0Gr7m7%2B0TJOLpMg9E5H9lx%2Fsax6ZUZ9snjS9Utqsak7nlbOmTDa3uh5ovXXjun2XCISg9TX8yY6RWWyQlSC1cSJ5K1DqVTH3Y4TQLTHuWlhoEOaiouXg63WH53VHaSV0lIVmzkFmHqt%2F2xexHYgRWQy8mhV1HkXDTXuMKJemcx8xlS0sai0JZDh6KnSq5QzvlNSP%2FOCauWMSsDUp2fyQQ6h7rs8B9E1CBdkdLQwlp2v4Xua%2BNAm8EUTc4Wx1WvTYPoi1H%2FZIGnTIF9dyBzSn%2B%2BMOKzdIdbQX5aLXV5vxyRTB29cHBSqL5XSJVBNiLy3XS27KbqOeeGd9L0msjW%2FKiWGfgmsKU3T5vtipGK5V6HIqe

In [ ]:
url = 

In [15]:
# Bicycle Parking

bicycle_parking_url = "http://datamall2.mytransport.sg/ltaodataservice/BicycleParkingv2"

bicycle_parking_api_response = api_query(api_link=bicycle_parking_url, agent_id="test", api_key=LTA_API_KEY)
print(bicycle_parking_url)
response_content = bicycle_parking_api_response.get("value")[0].get("Link")
response_content

404 Client Error: Not found. Verify that the request URL/Parameters are spelled correctly for url: http://datamall2.mytransport.sg/ltaodataservice/BicycleParkingv2
http://datamall2.mytransport.sg/ltaodataservice/BicycleParkingv2


AttributeError: 'NoneType' object has no attribute 'get'

In [6]:
## Platform crowd density real time

platform_crowd_density = "http://datamall2.mytransport.sg/ltaodataservice/PCDRealTime"
platform_crowd_density_real_time_data = api_query(api_link=platform_crowd_density, agent_id="test", api_key=LTA_API_KEY)
platform_crowd_density_real_time_data

404 Client Error: Not Found for url: http://datamall2.mytransport.sg/ltaodataservice/PCDRealTime
